In [1]:
import os
import csv
import json
import pandas as pd
import torch
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt

from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

import numpy as np
from sklearn.metrics import f1_score

import random
import warnings
warnings.filterwarnings('ignore')

In [2]:

def read_data(path):
    tsv_file = open(path)
    read_tsv = csv.reader(tsv_file, delimiter="\t")

    X = []
    Y = []
    i = 0
    print('Reading articles...')
    for row in read_tsv:
        #print(row)
        # Opening JSON file
        if(row[0][:]!='ID'):

            #if(i%1000==0):
            #    print('Reading article ', i)

            json_file = open('data/jsons/'+row[0][:]+'.json',)
            #print(row[1][:])
            # Reading JSON file
            data = json.load(json_file)

            Y.append(data['bias'])
            X.append(data['content'])


            #print(target)
            #print(text)

            # Closing file
            json_file.close()
            i = i + 1

    tsv_file.close()
    print('done')
    
    return X, Y


In [3]:
print('Load training dataset')
train_data_path = "data/splits/random/train.tsv"
X_train, Y_train = read_data(train_data_path)

# dictionary of lists 
dict_train = {'content': X_train, 'bias': Y_train} 
df_train = pd.DataFrame(dict_train)
#df_train.to_csv('data/splits/media/train_data.tsv', index=False)

print('Load validation dataset')
val_data_path = "data/splits/random/valid.tsv"
X_val, Y_val = read_data(val_data_path)

# dictionary of lists 
dict_val = {'content': X_val, 'bias': Y_val} 
df_val = pd.DataFrame(dict_val)
#df_val.to_csv('data/splits/media/val_data.tsv', index=False)

print('Load test dataset')
test_data_path = "data/splits/random/test.tsv"
X_test, Y_test = read_data(test_data_path)

# dictionary of lists 
dict_test = {'content': X_test, 'bias': Y_test} 
df_test = pd.DataFrame(dict_test)

#df_test.to_csv('data/splits/media/test_data.tsv', index=False)


Load training dataset
Reading articles...
done
Load validation dataset
Reading articles...
done
Load test dataset
Reading articles...
done


In [4]:
train  = df_train[['content', 'bias']] 
train.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

val = df_val[['content', 'bias']] 
val.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

test = df_test[['content', 'bias']]
test.columns = ['DATA_COLUMN', 'LABEL_COLUMN']

In [5]:
train.head()

,DATA_COLUMN,LABEL_COLUMN
0,Most taxpayers will get a check from the $ 2tn...,0
1,The backbone of President Obama ’ s health car...,2
2,Then-FBI Director James Comey testifying on Ca...,2
3,Fox News Has n't Always Shared Robert Driscoll...,1
4,LISTEN TO ARTICLE 2:15 SHARE THIS ARTICLE Shar...,1


In [6]:
val.head()

,DATA_COLUMN,LABEL_COLUMN
0,President Trump on Monday said he doesn ’ t be...,2
1,( CNN ) First he ducks into the shower of his ...,0
2,Conventional wisdom is that Obama can expect h...,0
3,CLOSE Saudi Arabia and Iran will no longer be ...,1
4,President Donald Trump on Saturday ripped repo...,2


In [7]:
test.head()

,DATA_COLUMN,LABEL_COLUMN
0,"For the last several days , the entire Western...",2
1,"Tomorrow morning , House Budget Chairman Paul ...",0
2,WASHINGTON ( ███ ) - President Donald Trump on...,1
3,"President Donald Trump arrives in Hanoi , Viet...",2
4,President Trump ’ s fiery response to the even...,2


In [8]:
train.LABEL_COLUMN.value_counts()

2    10240
0     9750
1     7988
Name: LABEL_COLUMN, dtype: int64

In [9]:
label_dict = {'right': 2, 'center': 1, 'left': 0}
label_dict

{'right': 2, 'center': 1, 'left': 0}

In [10]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [11]:
max_length = 128

In [14]:
encoded_data_train = tokenizer.batch_encode_plus(
    train.DATA_COLUMN.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=max_length,
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    val.DATA_COLUMN.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=max_length,
    truncation=True,
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    test.DATA_COLUMN.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=max_length,
    truncation=True,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train.LABEL_COLUMN.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(val.LABEL_COLUMN.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(test.LABEL_COLUMN.values)

In [15]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

dataset_test = TensorDataset(input_ids_test, 
                              attention_masks_test,
                              labels_test)

In [16]:
print(len(dataset_train))
print(len(dataset_val))
print(len(dataset_test))

27978
6996
1300


In [17]:
dataset_val.tensors

(tensor([[  101,  2343,  8398,  ...,  5975,  2071,   102],
         [  101,  1006, 13229,  ...,  1010,  4614,   102],
         [  101,  7511,  9866,  ...,  1996,  2317,   102],
         ...,
         [  101,  2899,  1006,  ...,  1012,  2129,   102],
         [  101,  2044,  1037,  ...,  2005,  2281,   102],
         [  101,  1996,  4450,  ...,  1998, 13712,   102]]),
 tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1]]),
 tensor([2, 0, 0,  ..., 0, 1, 1]))

In [18]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [19]:
batch_size = 16

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

dataloader_test = DataLoader(
    dataset_test,
    sampler=RandomSampler(dataset_test),
    batch_size=batch_size
)

In [20]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [21]:
epochs = 2

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

In [12]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [13]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [14]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [26]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [27]:
print(len(dataloader_train))
print(len(dataloader_val))
print(len(dataloader_test))

1749
438
82


In [28]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [38]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]49095690
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1749 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.7804814655484984


  0%|          | 0/438 [00:00<?, ?it/s]

Validation loss: 0.6391319493384666
F1 Score (weighted): 0.7164154275474955


Epoch 2:   0%|          | 0/1749 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5726364694062882


  0%|          | 0/438 [00:00<?, ?it/s]

Validation loss: 0.6027836488500306
F1 Score (weighted): 0.7451166800804621


## Training Set Evaluation

In [40]:
train_loss, train_predictions, true_trains = evaluate(dataloader_train)
train_f1 = f1_score_func(train_predictions, true_trains)
print(f'Test loss: {train_loss}')
print(f'Test F1 Score (weighted): {train_f1}')
accuracy_per_class(train_predictions, true_trains)

  0%|          | 0/1749 [00:00<?, ?it/s]

Test loss: 0.4907074200214148
Test F1 Score (weighted): 0.8031065133549401
Class: left
Accuracy:8325/9750

Class: center
Accuracy:5866/7988

Class: right
Accuracy:8294/10240



## Validation Set Evaluation

In [41]:
val_loss, val_predictions, true_vals = evaluate(dataloader_val)
val_f1 = f1_score_func(val_predictions, true_vals)
print(f'Test loss: {val_loss}')
print(f'Test F1 Score (weighted): {val_f1}')
accuracy_per_class(val_predictions, true_vals)

  0%|          | 0/438 [00:00<?, ?it/s]

Test loss: 0.6029245162459269
Test F1 Score (weighted): 0.7451166800804621
Class: left
Accuracy:1931/2438

Class: center
Accuracy:1363/1998

Class: right
Accuracy:1923/2560



## Test Set Evaluation 

In [39]:
test_loss, test_predictions, true_tests = evaluate(dataloader_test)
test_f1 = f1_score_func(test_predictions, true_tests)
print(f'Test loss: {test_loss}')
print(f'Test F1 Score (weighted): {test_f1}')
accuracy_per_class(test_predictions, true_tests)

  0%|          | 0/82 [00:00<?, ?it/s]

Test loss: 0.6778406751591984
Test F1 Score (weighted): 0.7052951796970037
Class: left
Accuracy:280/402

Class: center
Accuracy:278/299

Class: right
Accuracy:364/599



# Load trained model and predict on test dataset

In [16]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))
model_save_file = os.path.join('./Models/', 'BERT_ft_Epoch2.model')
model.load_state_dict(torch.load(model_save_file))
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [30]:
test_loss, test_predictions, true_tests = evaluate(dataloader_test)
test_f1 = f1_score_func(test_predictions, true_tests)
print(f'Test loss: {test_loss}')
print(f'Test F1 Score (weighted): {test_f1}')
accuracy_per_class(test_predictions, true_tests)

  0%|          | 0/82 [00:00<?, ?it/s]

Test loss: 0.6787759489039096
Test F1 Score (weighted): 0.7052951796970037
Class: left
Accuracy:280/402

Class: center
Accuracy:278/299

Class: right
Accuracy:364/599



# Explain the model using LIME

In [44]:
#https://github.com/marcotcr/lime/issues/409
#https://medium.com/@kalia_65609/interpreting-an-nlp-model-with-lime-and-shap-834ccfa124e4
#https://marcotcr.github.io/lime/tutorials/Lime%20-%20basic%20usage%2C%20two%20class%20case.html
#below one may be good
#https://stackoverflow.com/questions/64484738/applying-lime-interpretation-on-my-fine-tuned-bert-for-sequence-classification-m

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict
import torch.nn.functional as F

In [18]:
#model.cpu()

In [18]:
def predictor(texts):
    outputs = model(**tokenizer(texts, add_special_tokens=True, return_attention_mask=True, 
                                pad_to_max_length=True, max_length=128, 
                                truncation=True,return_tensors='pt').to(device))
    
    tensor_logits = outputs[0]
    
    probas = F.softmax(tensor_logits).cpu().detach().numpy()
    return probas


In [19]:
idx = 500
num_features = 30
class_names = ["left", "center", "right"]

text = val["DATA_COLUMN"][idx]
print(tokenizer(text, add_special_tokens=True, return_attention_mask=True, 
                pad_to_max_length=True, max_length=128, truncation=True, return_tensors='pt'))

explainer = LimeTextExplainer(class_names=class_names)


{'input_ids': tensor([[  101,  2637,  1521,  1055,  2327,  4454,  2880,  8969,  9857,  2008,
          2343,  8112,  1998,  2060,  3026,  2317,  2160,  4584,  2020,  2092,
          5204,  1997,  1057,  1012,  1055,  1012,  9867,  3450, 14126,  4177,
          1997,  5379,  3097,  3741,  1517,  1037,  9762, 26917,  1997,  1996,
          3447,  1521,  1055, 16021,  2378, 14505,  1999,  3522,  2420,  2008,
          1996,  2343,  2001, 11499,  1997,  2107, 22624,  1012,  2472,  1997,
          2120,  4454,  2508,  1054,  1012, 28618,  4842,  2649,  1996, 14126,
          1997,  3097,  4177,  1010,  2164,  2137,  6956,  1010,  2004,  1037,
          1523,  8050,  1524,  7814,  1997,  4454,  7215,  1010,  1998,  2056,
          4445,  1996,  9915,  4496,  1996,  2120,  3036,  4034,  2064, 11112,
          2046,  1037,  2445,  3003,  1521,  1055,  2797,  4806,  2302,  2317,
          2160, 15709,  1012,  2010, 10896,  1010,  2081,  2076,  1037,  2186,
          1997, 11937, 26243, 15800,  

In [20]:
logits = predictor(text)
print(logits)

[[0.0142169  0.06790356 0.9178796 ]]


In [22]:
exp = explainer.explain_instance(text, predictor, num_features=num_features)

RuntimeError: CUDA out of memory. Tried to allocate 1.83 GiB (GPU 0; 5.81 GiB total capacity; 4.09 GiB already allocated; 265.75 MiB free; 4.14 GiB reserved in total by PyTorch)

In [21]:
#torch.cuda.empty_cache()

print("Probability (Left) =", logits[0, 0])
print("Probability (Center) =", logits[0, 1])
print("Probability (Right) =", logits[0, 2])
print("True Class is:", class_names[val["LABEL_COLUMN"][idx]])

Probability (Left) = 0.0142169045
Probability (Center) = 0.067903556
Probability (Right) = 0.9178796
True Class is: right


In [47]:
exp.show_in_notebook(text=text)

NameError: name 'exp' is not defined

In [43]:
idx = 500
num_features = 30
explainer = LimeTextExplainer(class_names = class_names)
exp = explainer.explain_instance(val_df["content"][idx], c.predict_proba, num_features = num_features)

print("Article: \n", val_df["content"][idx])
print("Probability (Left) =", c.predict_proba([val_df["content"][idx]])[0, 0])
print("Probability (Center) =", c.predict_proba([val_df["content"][idx]])[0, 1])
print("Probability (Right) =", c.predict_proba([val_df["content"][idx]])[0, 2])
print("True Class is:", class_names[val_df["bias"][idx]])

TypeError: Last step of Pipeline should implement fit or be the string 'passthrough'. 'BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_f